# Week 8: EDA

## Setup

In [1]:
import pandas as pd
import plotly.express as px
import kaleido
import os
import numpy as np

In [2]:
mon = pd.read_csv("../data/monday.csv", index_col=0, parse_dates=True, sep=";")
tue = pd.read_csv("../data/tuesday.csv", index_col=0, parse_dates=True, sep=";")
wed = pd.read_csv("../data/wednesday.csv", index_col=0, parse_dates=True, sep=";")
thu = pd.read_csv("../data/thursday.csv", index_col=0, parse_dates=True, sep=";")
fri = pd.read_csv("../data/friday.csv", index_col=0, parse_dates=True, sep=";")


modify the customer_no to be unique throughout the days

In [3]:
dflist = [mon,tue,wed,thu,fri]

for i in range(4):
    dflist[i+1]["customer_no"] = dflist[i+1]["customer_no"] + dflist[i]["customer_no"].max()

In [4]:
df = pd.concat([mon,tue,wed,thu,fri])
#df.reset_index(inplace=True)

df

,customer_no,location
timestamp,,
2019-09-02 07:03:00,1,dairy
2019-09-02 07:03:00,2,dairy
2019-09-02 07:04:00,3,dairy
2019-09-02 07:04:00,4,dairy
2019-09-02 07:04:00,5,spices
...,...,...
2019-09-06 21:50:00,7435,dairy
2019-09-06 21:50:00,7442,checkout
2019-09-06 21:50:00,7443,checkout


In [28]:
df[df["customer_no"].between(1446,1447)]

,customer_no,location,day,day_name,hour,min
timestamp,,,,,,
2019-09-02 21:50:00,1446,dairy,2,Monday,21,50
2019-09-02 21:50:00,1447,fruit,2,Monday,21,50


create time related features for easier filtering

In [5]:
df["day"]  = df.index.day
df["day_name"] = df.index.day_name()
df["hour"] = df.index.hour
df["min"]  = df.index.minute
df

,customer_no,location,day,day_name,hour,min
timestamp,,,,,,
2019-09-02 07:03:00,1,dairy,2,Monday,7,3
2019-09-02 07:03:00,2,dairy,2,Monday,7,3
2019-09-02 07:04:00,3,dairy,2,Monday,7,4
2019-09-02 07:04:00,4,dairy,2,Monday,7,4
2019-09-02 07:04:00,5,spices,2,Monday,7,4
...,...,...,...,...,...,...
2019-09-06 21:50:00,7435,dairy,6,Friday,21,50
2019-09-06 21:50:00,7442,checkout,6,Friday,21,50
2019-09-06 21:50:00,7443,checkout,6,Friday,21,50


In [6]:
%store df

Stored 'df' (DataFrame)


## Q1: total number of customers in each section?

In [7]:
df.groupby("location").nunique()["customer_no"]

location
checkout    7417
dairy       3818
drinks      3214
fruit       4284
spices      2938
Name: customer_no, dtype: int64

## Q2, Q3: total number of customers in each section over time and total number of customers in checkout over time

In [8]:
plotdf = df.groupby([df["day"], df["day_name"],df["hour"], df["location"]]).nunique()["customer_no"].reset_index()
plotdf

,day,day_name,hour,location,customer_no
0,2,Monday,7,checkout,92
1,2,Monday,7,dairy,52
2,2,Monday,7,drinks,52
3,2,Monday,7,fruit,48
4,2,Monday,7,spices,41
...,...,...,...,...,...
370,6,Friday,21,checkout,52
371,6,Friday,21,dairy,31
372,6,Friday,21,drinks,28
373,6,Friday,21,fruit,28


plot

In [9]:
fig=px.line(
    plotdf,
    x='hour',
    y='customer_no',
    color='location',
    animation_frame='day_name',
    markers=True,
    labels={
        "customer_no": "total # of customers",
        "day_name": "weekday"
    },
    template='plotly_dark'
    )

fig.update_layout(autosize=False, width=1400, height=700)

fig.show()

if not os.path.exists("../plots"):
    os.mkdir("../plots")

if not os.path.exists("../plots/customers.html"):
    fig.write_html("../plots/customers.html")

## Q4 Time customer spent in market

In [10]:
dfdroptime = df.drop(columns=(["day", "day_name", "hour", "min"])).reset_index()

In [11]:
plotser = (dfdroptime.groupby("customer_no").max()["timestamp"]-dfdroptime.groupby("customer_no").min()["timestamp"]).apply(lambda x: x.seconds//60)
plotser = plotser[plotser > 0]

plot


In [12]:
fig = px.histogram(
    plotser,
    labels={
        "value": "minutes spent in market",
    },
    template='plotly_dark'
    )

fig.update_layout(showlegend=False, autosize=False, width=1400, height=700, yaxis_title="# of customers")

fig.show()

if not os.path.exists("../plots"):
    os.mkdir("../plots")

if not os.path.exists("../plots/time_spent.svg"):
    fig.write_image("../plots/time_spent.svg")


avg. customer stay time

In [13]:
plotser.mean()

6.247713824636902

## Calc for sim

### Customer Generation

mean and std of customers arriving per min for "normal" hours

In [14]:
mu = plotdf[(plotdf["location"]=="checkout") & (plotdf["hour"] != 8) & (plotdf["hour"] != 19)]["customer_no"].mean()/60
sigma = plotdf[(plotdf["location"]=="checkout") & (plotdf["hour"] != 8) & (plotdf["hour"] != 19)]["customer_no"].std()/60
mu, sigma

(1.5346153846153847, 0.3406782921143475)

mean and std of customers arriving per min for "peak" hours

In [15]:
mu_peak = plotdf[(plotdf["location"]=="checkout") & ((plotdf["hour"] == 8) | (plotdf["hour"] == 19))]["customer_no"].mean()/60
sigma_peak = plotdf[(plotdf["location"]=="checkout") & ((plotdf["hour"] == 8) | (plotdf["hour"] == 19))]["customer_no"].std()/60
mu_peak, sigma_peak

(2.3866666666666663, 0.41334229380968485)

**better idea: create a ML model for customers gen per min**

### Customer stay duration

In [16]:
# draft idea
np.mean(np.random.exponential(plotser.mean(), 100000))

6.238081043458113

### Arrival aisle placement

In [22]:
#arrivaldf = dfdroptime[dfdroptime["location"] != "checkout"]
arrivaldf = dfdroptime
arrivaldf

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices
...,...,...,...
24872,2019-09-06 21:50:00,7435,dairy
24873,2019-09-06 21:50:00,7442,checkout
24874,2019-09-06 21:50:00,7443,checkout
24875,2019-09-06 21:50:00,7444,drinks


In [23]:
firstarrivaldf = arrivaldf.groupby('customer_no').agg({'timestamp':'min', 'location':'first'})
firstarrivaldf[firstarrivaldf["location"]=="checkout"]

,timestamp,location
customer_no,,


In [24]:
arrival_probs = firstarrivaldf["location"].value_counts()/firstarrivaldf.shape[0]
arrival_probs

location
fruit     0.377435
dairy     0.287576
spices    0.181464
drinks    0.153526
Name: count, dtype: float64

In [ ]:
#sanity check
arrival_probs.sum()

1.0

In [ ]:
list(arrival_probs.index)

['fruit', 'dairy', 'spices', 'drinks']

In [ ]:
list(arrival_probs)

[0.3774345198119543,
 0.2875755540631296,
 0.18146406984553393,
 0.15352585627938214]

### Number of aisle switches

In [33]:
aisle_switch = df.groupby("customer_no").nunique()["min"]
aisle_switch = aisle_switch[aisle_switch > 1]

In [36]:
fig = px.histogram(
    aisle_switch,
    labels={
        "value": "# of aisle switches",
    },
    template='plotly_dark'
    )

fig.update_layout(showlegend=False, autosize=False, width=1400, height=700, yaxis_title="# of customers")

fig.show()

if not os.path.exists("../plots"):
    os.mkdir("../plots")

if not os.path.exists("../plots/aisles_switched.svg"):
    fig.write_image("../plots/aisles_switched.svg")

In [37]:
aisle_switch[aisle_switch == 18]

customer_no
526    18
Name: min, dtype: int64

In [38]:
df[df["customer_no"]==526]

,customer_no,location,day,day_name,hour,min
timestamp,,,,,,
2019-09-02 13:04:00,526,fruit,2,Monday,13,4
2019-09-02 13:06:00,526,dairy,2,Monday,13,6
2019-09-02 13:07:00,526,drinks,2,Monday,13,7
2019-09-02 13:08:00,526,spices,2,Monday,13,8
2019-09-02 13:09:00,526,fruit,2,Monday,13,9
2019-09-02 13:11:00,526,spices,2,Monday,13,11
2019-09-02 13:12:00,526,drinks,2,Monday,13,12
2019-09-02 13:14:00,526,spices,2,Monday,13,14
2019-09-02 13:15:00,526,dairy,2,Monday,13,15


In [39]:
aisle_switch.mean()

3.3442711135018826

probabylity to change location per min

In [41]:
1/(plotser.mean()/aisle_switch.mean())

0.5352791768909552